In [13]:
import numpy as np
import sympy as sym

In [14]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [15]:
def GetHermiteRecursive(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = ((2*x)*GetHermiteRecursive(n-1,x))-((2*(n-1))*GetHermiteRecursive(n-2,x))
   
    return sym.expand(poly,x)

In [16]:
GetHermiteRecursive(3,x)

8*x**3 - 12*x

In [17]:
def GetDHermite(n,x):
    Pn = GetHermiteRecursive(n,x)
    return sym.diff(Pn,x,3)

In [18]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
            xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [19]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [20]:
def GetAllRootsGHer(n):

    xn = np.linspace(-np.sqrt(4*n+1),np.sqrt(4*n+1),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermiteRecursive(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    #f len(Roots) != n:
     #   ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [23]:
GetAllRootsGHer(4)

array([], dtype=float64)

In [22]:
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    DHermite = []
    
    for i in range(n+1):
        DHermite.append(GetDHermite(i,x))
    
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Weights = 2**(n-1)*np.math.factorial(n)*np.sqrt(np.pi)/((n**2)*GetHermiteRecursive(n-1,Roots))
    
    return Weights